Please note that I'd usually first screen the data via tools like ydata-profiling, which is a check not performed in this sample code.

# Module imports

In [17]:
import os
import pandas as pd
import sys
import yaml

from sklearn.model_selection import train_test_split


In [18]:
# Add the src folder to the path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

In [19]:
import utils.data_validation as utils_dataval

In [20]:
with open('../config.yaml', 'r') as file:
    config = yaml.safe_load(file)


# Constants

In [21]:
TRAIN_PROP = 0.8
RANDOM_STATE = 42

# Load data

In [22]:
df = pd.read_csv('../data/model_df.csv')

In [23]:
df.head()

,visitor_id,query_id,destination_id,device,language,product_code,product_category,duration,ctr_14d,ctr_30d,clicked
0,e20b98b6-f567-4780-b794-b764cb8f817a,0003S-UcYv-V9CH,7,mobile,fr,7-34,4,263,0.088589,0.086260,0
1,e20b98b6-f567-4780-b794-b764cb8f817a,0003S-UcYv-V9CH,7,mobile,fr,7-11,3,187,0.091653,0.098138,0
2,e20b98b6-f567-4780-b794-b764cb8f817a,0003S-UcYv-V9CH,7,mobile,fr,7-9,10,131,0.079760,0.080651,0
3,e20b98b6-f567-4780-b794-b764cb8f817a,0003S-UcYv-V9CH,7,mobile,fr,7-19,10,253,0.094435,0.094269,1
4,e20b98b6-f567-4780-b794-b764cb8f817a,0003S-UcYv-V9CH,7,mobile,fr,7-33,5,242,0.108431,0.118036,0


# Process data

In [24]:
# Create mean-encoded variables based on device and product_category.
# Done as device is static within query. Columns are only useful to LGBM Ranker models if there are variations within a query.

def create_mean_encoded_variables(df, categorical_column, target_column):
    return df.groupby(categorical_column)[target_column].transform('mean')

df["device_prodcat_mean_encoded"] = create_mean_encoded_variables(df, ["device", "product_category"], "clicked")
df["language_prodcat_mean_encoded"] = create_mean_encoded_variables(df, ["language", "product_category"], "clicked")

In [25]:
df["product_category"] = df["product_category"].astype("category")

# Validate data

Please note that only a small amount of tests are written and I have not provided a full test suite that includes tests cases due to time constraints.

In [26]:
utils_dataval.validate_categorical_columns(df, config["categorical_cols"])

Validating column 'product_category'
Column 'product_category' is categorical (dtype: category)
All columns are categorical.


In [27]:
utils_dataval.validate_float_columns(df, config["float_cols"])

Validating column 'ctr_14d'
Validating column 'ctr_30d'
Validating column 'device_prodcat_mean_encoded'
Validating column 'language_prodcat_mean_encoded'
All columns are float.


In [28]:
column_range_dict = {"ctr_14d": {"min": 0, "max": 1}, "ctr_30d": {"min": 0, "max": 1}} # Just some examples
utils_dataval.validate_float_range(df, column_range_dict)

Validating column 'ctr_14d'
Validating column 'ctr_30d'
All columns are float and within range.


In [29]:
utils_dataval.validate_no_nulls(df, config["float_cols"] + config["integer_cols"] + config["categorical_cols"])

Validating column 'ctr_14d'
Validating column 'ctr_30d'
Validating column 'device_prodcat_mean_encoded'
Validating column 'language_prodcat_mean_encoded'
Validating column 'duration'
Validating column 'product_category'
All columns do not contain nulls.


# Split Data

In [30]:
# For simplicity, randomly split the data into training, validation, and test sets by query_id.
# Get unique query_ids
unique_query_ids = df['query_id'].unique()

# Split query_ids into train (80%), val (10%), and test (10%)
# First split: 80% train, 20% temp (which will be split into val and test)
train_query_ids, temp_query_ids = train_test_split(
    unique_query_ids, 
    test_size=0.2, 
    random_state=RANDOM_STATE
)

# Second split: split the 20% temp into 10% val and 10% test
val_query_ids, test_query_ids = train_test_split(
    temp_query_ids, 
    test_size=0.5, 
    random_state=RANDOM_STATE
)

# Create train, val, and test dataframes based on query_id splits
train_df = df[df['query_id'].isin(train_query_ids)].copy()
val_df = df[df['query_id'].isin(val_query_ids)].copy()
test_df = df[df['query_id'].isin(test_query_ids)].copy()

print(f"Total queries: {len(unique_query_ids)}")
print(f"Train queries: {len(train_query_ids)} ({len(train_query_ids)/len(unique_query_ids)*100:.1f}%)")
print(f"Val queries: {len(val_query_ids)} ({len(val_query_ids)/len(unique_query_ids)*100:.1f}%)")
print(f"Test queries: {len(test_query_ids)} ({len(test_query_ids)/len(unique_query_ids)*100:.1f}%)")


Total queries: 10000
Train queries: 8000 (80.0%)
Val queries: 1000 (10.0%)
Test queries: 1000 (10.0%)


In [31]:
train_df = train_df[config["non_feature_cols"] + config["float_cols"] + config["integer_cols"] + config["categorical_cols"] + [config["target_col"]]]
val_df = val_df[config["non_feature_cols"] + config["float_cols"] + config["integer_cols"] + config["categorical_cols"] + [config["target_col"]]]
test_df = test_df[config["non_feature_cols"] + config["float_cols"] + config["integer_cols"] + config["categorical_cols"] + [config["target_col"]]]

In [32]:
# One final change to ensure train_df, val_df, and test_df are all sorted by query_id.
# Not a robust solution but probably good enough for now.

train_df = train_df.sort_values(by="query_id")
val_df = val_df.sort_values(by="query_id")
test_df = test_df.sort_values(by="query_id")

# Save data

In [33]:
# Save model_df to data/model_df.csv. Parquet likely a better choice, but using csv for simplicity and to reduce dependencies.
train_df.to_csv('../data/train_df.csv', index=False)
val_df.to_csv('../data/val_df.csv', index=False)
test_df.to_csv('../data/test_df.csv', index=False)